In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow as tf

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0:1], 'GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass

In [3]:
def multiple_gradients(x, y, point = 1, activation = tf.keras.activations.tanh, learning_rate = 0.01):
    x = tf.convert_to_tensor(x, dtype=tf.float64)
    y = tf.convert_to_tensor(y, dtype=tf.float64)


    model = tf.keras.Sequential([
    layers.Dense(100, activation=activation, input_shape=[1,]),
    layers.Dropout(0.5), 
    layers.Dense(100, activation=activation),
    layers.Dropout(0.5),
    layers.Dense(100, activation=activation),
    layers.Dense(1)
    ])

    model.compile(
        run_eagerly=False,
        optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
        loss=tf.keras.losses.mean_squared_error,
    )

    model.fit(x, y, epochs=20, batch_size=128)


    X = tf.convert_to_tensor([[point]], dtype=tf.float64)

    with tf.GradientTape() as tapef3:
        tapef3.watch(X)
        with tf.GradientTape(persistent = True) as tapef2:
            tapef2.watch(X)
            with tf.GradientTape(persistent = True) as tapef1:
                tapef1.watch(X)
                y = model(X)
#                 yprime = tapef1.gradient(y,X)
    #             print("predicted: ", y, "actual: ", tf.math.sin(X))
            grad1 = tapef1.gradient(y, X)
#             print("gradient1: ", grad1)
        grad2 = tapef2.gradient(grad1, X)
#         print("gradient2: ", grad2)
    grad3 = tapef3.gradient(grad2,X)
#     print("gradient3: ", grad3)

    return model(X).numpy(), grad1.numpy(), grad2.numpy(), grad3.numpy()

# print(multiple_gradients(x, y, point = 1, activation = tf.keras.activations.tanh, learning_rate = 0.01))

In [4]:
def grad(x, y, point=1):
    x_min = np.min(x)
    x_max = np.max(x)
    x_len = len(x)
    x = tf.convert_to_tensor(x, dtype=tf.float64)
    y = tf.convert_to_tensor(y, dtype=tf.float64)

    model = tf.keras.Sequential([
    layers.Dense(100, activation=tf.keras.activations.relu, input_shape=[1,]),
    layers.Dropout(0.5),   
    layers.Dense(100, activation=tf.keras.activations.relu),
    layers.Dropout(0.5),
    layers.Dense(100, activation=tf.keras.activations.relu),
    layers.Dense(1)
    ])

    # compile sets the training parameters
    model.compile(
        run_eagerly=False,
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
        loss=tf.keras.losses.mean_squared_error,
    )

    model.fit(x, y, epochs=20, batch_size=128)

    X_point = tf.convert_to_tensor([[point]], dtype=tf.float64)
    X_all = tf.linspace(x_min, x_max, x_len, )

    with tf.GradientTape(persistent = True) as tapef1:
        tapef1.watch(X_all)
        y = model(X_all)
    grad_all = tapef1.gradient(y, X_all)

    with tf.GradientTape() as tape:
      tape.watch(X_point)
      y = model(X_point)
    grad_point = tape.gradient(y, X_point)

    return grad_point, grad_all, X_all, model(X_point)

def three_grads(x, y, point = 1):
    grad_point_a, grad_all_a, X_all_a, y_point = grad(x,y, point = point)
    grad_point_b, grad_all_b, X_all_b, _ = grad(X_all_a,grad_all_a)
    grad_point_c, grad_all_c, X_all_c, _ = grad(X_all_b,grad_all_b)
    return y_point.numpy(), grad_point_a.numpy(), grad_point_b.numpy(), grad_point_c.numpy()

# print(three_grads(x, y, 1))

In [5]:
x = np.random.rand(100000)*2+1
functions = [np.sin(x), np.cos(x), np.sqrt(x), x**2, x**3, np.exp(x), np.log(x)]
name = ["np.sin(x)", "np.cos(x)", "np.sqrt(x)", "x**2", "x**3", "np.exp(x)", "np.log(x)"]
point = 1

all_results = []
for y in range(len(functions)):
    results = []
    for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
        results.append(np.array(multiple_gradients(x, functions[y], point = point, activation = i, learning_rate = 0.01)).reshape(4))
    results.append(np.array(three_grads(x, functions[y], point = point)).reshape(4))   
#     np.savetxt("comparing_grad_methods_%s.csv" %name[y], np.asarray(results), delimiter=",")
    all_results.append(results)

Epoch 1/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0801
Epoch 2/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0673
Epoch 3/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0649
Epoch 4/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0639
Epoch 5/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0636
Epoch 6/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0633
Epoch 7/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0634
Epoch 8/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0633
Epoch 9/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0632
Epoch 10/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0632
Epoch 11/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0630
Epoch 12/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0631
Epoch 13/20
782/782 [====

782/782 [==============================] - 1s 1ms/step - loss: 0.0511
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0264
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0197
Epoch 4/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0159
Epoch 5/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0135
Epoch 6/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0122
Epoch 7/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0112
Epoch 8/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0106
Epoch 9/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0100
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0095
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0093
Epoch 12/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0091
Epoch 13/20
782/782 [===============

782/782 [==============================] - 1s 2ms/step - loss: 0.2318
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2286
Epoch 13/20
782/782 [===============

782/782 [==============================] - 1s 2ms/step - loss: 0.2967
Epoch 2/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0457
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0436
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0428
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0419
Epoch 6/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0414
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0410
Epoch 8/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0405
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0397
Epoch 10/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0395
Epoch 11/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0391
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0384
Epoch 13/20
782/782 [===============

782/782 [==============================] - 1s 1ms/step - loss: 0.0105
Epoch 2/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0029
Epoch 3/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0016
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0011
Epoch 5/20
782/782 [==============================] - 1s 1ms/step - loss: 7.6473e-04
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 5.8448e-04
Epoch 7/20
782/782 [==============================] - 1s 1ms/step - loss: 4.5546e-04
Epoch 8/20
782/782 [==============================] - 1s 1ms/step - loss: 3.7158e-04
Epoch 9/20
782/782 [==============================] - 1s 1ms/step - loss: 3.0484e-04
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 2.5682e-04
Epoch 11/20
782/782 [==============================] - 1s 1ms/step - loss: 2.2027e-04
Epoch 12/20
782/782 [==============================] - 1s 1ms/step - loss: 1.8939e-04
Epoc

782/782 [==============================] - 1s 2ms/step - loss: 5.4205
Epoch 1/20
782/782 [==============================] - 1s 1ms/step - loss: 0.8711
Epoch 2/20
782/782 [==============================] - 1s 1ms/step - loss: 0.3639
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2973
Epoch 4/20
782/782 [==============================] - 1s 1ms/step - loss: 0.2602
Epoch 5/20
782/782 [==============================] - 1s 1ms/step - loss: 0.2375
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2222
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2113
Epoch 8/20
782/782 [==============================] - 1s 1ms/step - loss: 0.2043
Epoch 9/20
782/782 [==============================] - 1s 1ms/step - loss: 0.2001
Epoch 10/20
782/782 [==============================] - 1s 1ms/step - loss: 0.1883
Epoch 11/20
782/782 [==============================] - 1s 1ms/step - loss: 0.1832
Epoch 12/20
782/782 [================

782/782 [==============================] - 1s 2ms/step - loss: 0.8214
Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 13.5359
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 2.2784
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 1.3811
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 1.1117
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 0.8218
Epoch 6/20
782/782 [==============================] - 1s 1ms/step - loss: 0.7388
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.6941
Epoch 8/20
782/782 [==============================] - 1s 1ms/step - loss: 0.6238
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.5870
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.5157
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4845
Epoch 12/20
782/782 [===============

782/782 [==============================] - 1s 1ms/step - loss: 1.1138e-04
Epoch 19/20
782/782 [==============================] - 1s 1ms/step - loss: 1.0032e-04
Epoch 20/20
782/782 [==============================] - 1s 2ms/step - loss: 8.9873e-05
Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 22.1372
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 7.2809
Epoch 3/20
782/782 [==============================] - 1s 1ms/step - loss: 3.6301
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 2.7657
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 2.4567
Epoch 6/20
782/782 [==============================] - 1s 1ms/step - loss: 2.1956
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 2.0420
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 1.9358
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 1.7981
Epoch 10/20
782/782 [===

782/782 [==============================] - 1s 1ms/step - loss: 0.3058
Epoch 19/20
782/782 [==============================] - 1s 1ms/step - loss: 0.2970
Epoch 20/20
782/782 [==============================] - 1s 1ms/step - loss: 0.2888
Epoch 1/20
782/782 [==============================] - 1s 1ms/step - loss: 0.8947
Epoch 2/20
782/782 [==============================] - 1s 1ms/step - loss: 0.5381
Epoch 3/20
782/782 [==============================] - 1s 1ms/step - loss: 0.4886
Epoch 4/20
782/782 [==============================] - 1s 1ms/step - loss: 0.4672
Epoch 5/20
782/782 [==============================] - 1s 1ms/step - loss: 0.4518
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4370
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4268
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4130
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.3983
Epoch 10/20
782/782 [================

782/782 [==============================] - 1s 1ms/step - loss: 0.0025
Epoch 19/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0024
Epoch 20/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0024
Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 0.1086
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0948
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0948
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0948
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0948A: 0s - loss: 0.
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0948
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0948
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0948
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0948
Epoch 10/20
782/782 [

In [6]:
#comparing results for sinx
# ["np.sin(x)", "np.cos(x)", "np.sqrt(x)", "x**2", "x**3", "np.exp(x)", "np.log(x)"]

from sklearn.metrics import mean_squared_error

#sin
print("sine")
for i in all_results[0]:
    correct = [np.sin(1), np.cos(1), -np.sin(1), -np.cos(1)]
    try: 
        print(mean_squared_error(i, correct))
    except:
        print("nan")

#cos
print("cosine")
for i in all_results[1]:
    correct = [np.cos(1), -np.sin(1), -np.cos(1), np.sin(1)]
    try:
        print(mean_squared_error(i, correct))
    except:
        print("nan")

#sqrt
print("sqrt")
for i in all_results[2]:
    correct = [np.sqrt(1), 1/2/np.sqrt(1), -1/(4*1**(1.5)), 0.375/(1**2.5)]
    try:
        print(mean_squared_error(i, correct))
    except:
        print("nan")  
    
#square
print("square")
for i in all_results[3]:
    correct = [1, 2, 2, 0]
    try:     
        print(mean_squared_error(i, correct))
    except:
        print("nan")


#cube
print("cube")
for i in all_results[4]:
    correct = [1, 3, 3, 0]
    try: 
        print(mean_squared_error(i, correct))
    except:
        print("nan")
    
#exponential
print("exponential")
for i in all_results[5]:
    correct = [np.exp(1), np.exp(1), np.exp(1), np.exp(1)]
    try: 
        print(mean_squared_error(i, correct))
    except:
        print("nan")
    
#ln
print("ln")
for i in all_results[6]:
    correct = [0, 1, -1/1, 2/1]
    try:
        print(mean_squared_error(i, correct))
    except:
        print("nan")

sine
0.32484957833990336
0.3405432653265929
0.36050230350005646
0.32438215725727376
0.31145273339321083
cosine
0.6198449265639111
0.3008848473849034
0.15200658578152398
0.6272268056960371
6.493923777413157
sqrt
0.15076044568872482
0.1115256132717066
0.07822782262071099
0.15289139494831777
0.1161495671705286
square
8.518339631275477
7.240870337269545
2.0260028004215833
4.766212973144126
2.07223208679432
cube
70.72968842556286
9.002748765720142
37.66928164881047
25.234532992633408
22.610823710073415
exponential
12.144571182744253
15.11923991679065
1.8385775639886093
14.673984315723933
5.649217328513431
ln
1.6061985814814475
1.4252941549988156
1.9371915893184952
1.604877726267166
1.5099846295113046


In [8]:
#comparing results for sinx
# ["np.sin(x)", "np.cos(x)", "np.sqrt(x)", "x**2", "x**3", "np.exp(x)", "np.log(x)"]

def maape(actual, pred):
    return np.mean(np.arctan(np.abs((actual - pred) / pred))) * 100

#sin
print("sine")
for i in all_results[0]:
    correct = [np.sin(1), np.cos(1), -np.sin(1), -np.cos(1)]
    try: 
        print(maape(i, correct))
    except:
        print("nan")

#cos
print("cosine")
for i in all_results[1]:
    correct = [np.cos(1), -np.sin(1), -np.cos(1), np.sin(1)]
    try:
        print(maape(i, correct))
    except:
        print("nan")

#sqrt
print("sqrt")
for i in all_results[2]:
    correct = [np.sqrt(1), 1/2/np.sqrt(1), -1/(4*1**(1.5)), 0.375/(1**2.5)]
    try:
        print(maape(i, correct))
    except:
        print("nan")  
    
#square
print("square")
for i in all_results[3]:
    correct = [1, 2, 2, 0]
    try:     
        print(maape(i, correct))
    except:
        print("nan")


#cube
print("cube")
for i in all_results[4]:
    correct = [1, 3, 3, 0]
    try: 
        print(maape(i, correct))
    except:
        print("nan")
    
#exponential
print("exponential")
for i in all_results[5]:
    correct = [np.exp(1), np.exp(1), np.exp(1), np.exp(1)]
    try: 
        print(maape(i, correct))
    except:
        print("nan")
    
#ln
print("ln")
for i in all_results[6]:
    correct = [0, 1, -1/1, 2/1]
    try:
        print(maape(i, correct))
    except:
        print("nan")

sine
60.107120849925764
61.73604268831239
68.03465955957681
61.12278120443524
60.35642694649388
cosine
84.2705975824831
59.93147678080197
30.984096455088817
84.59524593674462
81.77664760532932
sqrt
68.15041778870781
62.759460998588054
50.35918779302927
68.375379699787
61.67212006188175
square
61.584045094257256
79.88324270506189
64.95929576172172
110.50893726478017
89.03283085414151
cube
72.69171790902229
75.40148247785932
69.169115066754
115.07554783522447
114.89072647971064
exponential
52.59982530463178
61.60591593984772
44.20072844165949
87.60849716126455
64.41594369345789
ln
98.0879033119492
94.60341407220004
94.16351864645726
98.17476994899248
98.18071430944221


/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """
